In [ ]:
from collections import OrderedDict
from conllu import parse
from enum import Enum
import stanza
import os

In [2]:
stanza_nlp = stanza.Pipeline(lang='uk')

2020-05-03 18:20:54 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-05-03 18:20:54 INFO: Use device: cpu
2020-05-03 18:20:54 INFO: Loading: tokenize
2020-05-03 18:20:54 INFO: Loading: mwt
2020-05-03 18:20:54 INFO: Loading: pos
2020-05-03 18:20:56 INFO: Loading: lemma
2020-05-03 18:20:56 INFO: Loading: depparse
2020-05-03 18:20:57 INFO: Done loading processors!


In [3]:
script_path = os.path.abspath('__file__') 
path_list = script_path.split(os.sep)
script_directory = path_list[0:len(path_list)-5]
rel_path = "UD_Ukrainian-IU"
PATH = "/".join(script_directory[:4]) + "/" + rel_path
 
with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    train_trees = parse(f.read())

with open(PATH + "/uk_iu-ud-dev.conllu", "r") as f:
    test_trees = parse(f.read())


In [4]:
corpus = ["Цінова лихоманка у царині енергоресурсів засвідчує, що світова економіка переживає не найкращі часи.", 
          "Проте усвідомлення цього факту не завадило верховному представнику ЄС у закордонних справах та політиці безпеки Жозепу Боррелю під час спілкування з Дмитром Кулебою наголосити на зацікавленості Європейського Союзу у співпраці з Україною та готовності здійснити відкладений візит до України тільки-но для цього з’явиться можливість.",
          "Нагадаю, що пан Боррель мав намір відвідати Донбас, що протягом останніх років стало ледь не обов’язковою частиною для перебування в Україні поважних іноземних гостей.",
          "Міністри закордонних справ країн ЄС також продемонстрували готовність до розвитку співпраці з Україною, а Європейська Комісія оголосила про готовність виділити для України 1,2 мільярди для боротьби з наслідками коронавірусу.",  
          "Російська Федерація також демонструє активність щодо України, проте дещо іншого порядку.", 
          "У Москві свідомо ігнорують очевидне: словосполучення «російський гуманітарний конвой» з 2014 року стало синонімом незаконного втручання та підтримки бойовиків."]

In [5]:
def transform_sent(sentence):
    transformed_tree = []
    doc = stanza_nlp(sentence)
    for sent in doc.sentences:
        for word in sent.words:
            transformed_tree.append(OrderedDict([('id', int(word.id)), ('form',  word.text), ('lemma',word.lemma),
                                     ('upostag', word.upos), ('xpostag', None),
                                     ('feats', word.feats), ('head', int(word.head)), ('deprel', None),
                                     ('deps', None), ('misc', None)]))
    return transformed_tree

In [6]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"
    
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [7]:
def extract_features(stack, queue):
    features = dict()

    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
         # фічі для ост-го слова в стеці
        if stack_top["feats"] != None:
            features["s0-feats"] = str(stack_top["feats"])
        else:
            features["s0-feats"] = False
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
        if stack[-2]["feats"] != None:
            features["s1-feats"] = str(stack[-2]["feats"])
        else:
            features["s1-feats"] = False
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        # фічі для 1-го слова в черзі
        if queue_top["feats"] != None:
            features["q0-feats"] = str(queue_top["feats"])
        else:
            features["q0-feats"] = False
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        # фічі для 2-го слова в черзі
        if queue_next["feats"] != None:
            features["q1-feats"] = str(queue_next["feats"])
        else:
            features["q1-feats"] = False
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

In [8]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

In [9]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []
    
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                       queue[0] if len(queue) > 0 else None,
                       relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [10]:
for sentence in corpus:
    tree = transform_sent(sentence)
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])

    print("Number of words:", len(tree))
    print("Number of actions:", len(tree_labels))
    print("List of actions taken:", tree_labels)
    print("Features:")
    for word in tree_features:
        print(word) 
    print("---------------------------------------")

Number of words: 15
Number of actions: 31
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'left', 'right', 'shift', 'shift', 'shift', 'left', 'shift', 'left', 'left', 'left', 'right', 'shift', 'left', 'shift', 'left', 'right', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 's0-feats': False, 'q0-word': 'Цінова', 'q0-lemma': 'ціновий', 'q0-tag': 'ADJ', 'q0-feats': 'Case=Nom|Gender=Fem|Number=Sing', 'q1-word': 'лихоманка', 'q1-tag': 'NOUN', 'q1-feats': 'Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing', 'q2-tag': 'ADP', 'q3-tag': 'NOUN'}
{'s0-word': 'Цінова', 's0-lemma': 'ціновий', 's0-tag': 'ADJ', 's0-feats': 'Case=Nom|Gender=Fem|Number=Sing', 's1-tag': 'ROOT', 's1-feats': False, 'q0-word': 'лихоманка', 'q0-lemma': 'лихоманка', 'q0-tag': 'NOUN', 'q0-feats': 'Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing', 'q1-word': 'у', 'q1-tag': 'ADP', 'q1-feats': 'Case=L

Number of words: 47
Number of actions: 95
List of actions taken: ['shift', 'shift', 'shift', 'left', 'right', 'shift', 'left', 'reduce', 'left', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'shift', 'left', 'left', 'reduce', 'right', 'shift', 'left', 'right', 'right', 'right', 'right', 'shift', 'left', 'shift', 'right', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'reduce', 'left', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'reduce', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'shift', 'shift', 'shift', 'shift', 'left', 'shift', 'left', 'left', 'left', 'left', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'right', 'reduce', 'right', 'reduce', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 's0-feats

Number of words: 27
Number of actions: 55
List of actions taken: ['right', 'shift', 'shift', 'shift', 'right', 'reduce', 'left', 'left', 'left', 'right', 'right', 'right', 'right', 'shift', 'shift', 'shift', 'shift', 'left', 'left', 'shift', 'left', 'left', 'left', 'right', 'shift', 'shift', 'left', 'left', 'shift', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'shift', 'shift', 'left', 'left', 'reduce', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 's0-feats': False, 'q0-word': 'Нагадаю', 'q0-lemma': 'нагадати', 'q0-tag': 'VERB', 'q0-feats': 'Aspect=Perf|Mood=Ind|Number=Sing|Person=1|Tense=Fut|VerbForm=Fin', 'q1-word': ',', 'q1-tag': 'PUNCT', 'q1-feats': False, 'q2-tag': 'SCONJ', 'q3-tag': 'NOUN'}
{'s0-word': 'Нагадаю', 's0-lemma': 'нагадати', 's0-tag': 'VERB', 's0-feats': 'Aspect=Perf|Mood=Ind|Number=Sing|Person=1

Number of words: 33
Number of actions: 67
List of actions taken: ['shift', 'shift', 'left', 'right', 'right', 'right', 'shift', 'left', 'reduce', 'reduce', 'reduce', 'left', 'right', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'shift', 'shift', 'shift', 'left', 'shift', 'left', 'left', 'left', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'shift', 'shift', 'left', 'right', 'reduce', 'left', 'reduce', 'right', 'shift', 'left', 'reduce', 'right', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 's0-feats': False, 'q0-word': 'Міністри', 'q0-lemma': 'міністр', 'q0-tag': 'NOUN', 'q0-feats': 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur', 'q1-word': 'закордонних', 'q1-tag': 'ADJ', 'q1-feats': 'Case=Gen|Number=Plur', 'q2-tag': 'NOUN', 'q3-tag': 'NOUN'}


Number of words: 23
Number of actions: 47
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'left', 'right', 'right', 'shift', 'shift', 'shift', 'shift', 'shift', 'left', 'left', 'left', 'right', 'right', 'shift', 'shift', 'left', 'left', 'shift', 'left', 'reduce', 'reduce', 'left', 'left', 'reduce', 'right', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 's0-feats': False, 'q0-word': 'У', 'q0-lemma': 'у', 'q0-tag': 'ADP', 'q0-feats': 'Case=Loc', 'q1-word': 'Москві', 'q1-tag': 'PROPN', 'q1-feats': 'Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing', 'q2-tag': 'ADV', 'q3-tag': 'VERB'}
{'s0-word': 'У', 's0-lemma': 'у', 's0-tag': 'ADP', 's0-feats': 'Case=Loc', 's1-tag': 'ROOT', 's1-feats': False, 'q0-word': 'Москві', 'q0-lemma': 'Москва', 'q0-tag': 'PROPN', 'q0-feats': 'Animacy=Inan|Case=Loc|Gender=F

In [11]:
train_features, train_labels = [], []
for tree in train_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [12]:
test_features, test_labels = [], []
for sentence in corpus:
    tree = transform_sent(sentence)
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

322 322


In [13]:
from sklearn.feature_extraction import DictVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [14]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)
print("\nTotal number of features:", len(vec.get_feature_names()))


Total number of features: 115315


In [15]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)
print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

190298 322


In [16]:
lrc = LogisticRegression(random_state=42, solver="saga", multi_class="multinomial", max_iter=1000, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 646 epochs took 161 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='saga', tol=0.0001, verbose=1,
                   warm_start=False)

In [17]:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.96      0.83      0.89        83
      reduce       0.82      0.95      0.88        81
       right       0.82      0.84      0.83        75
       shift       0.92      0.88      0.90        83

    accuracy                           0.88       322
   macro avg       0.88      0.88      0.88       322
weighted avg       0.88      0.88      0.88       322

